In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix
import collections

In [2]:
df = pd.read_csv('credit_dataset.csv')
df.head()

,credit.rating,account.balance,credit.duration.months,previous.credit.payment.status,credit.purpose,credit.amount,savings,employment.duration,installment.rate,marital.status,...,residence.duration,current.assets,age,other.credits,apartment.type,bank.credits,occupation,dependents,telephone,foreign.worker
0,1,1,18,3,2,1049,1,1,4,1,...,4,2,21,2,1,1,3,1,1,1
1,1,1,9,3,4,2799,1,2,2,3,...,2,1,36,2,1,2,3,2,1,1
2,1,2,12,2,4,841,2,3,2,1,...,4,1,23,2,1,1,2,1,1,1
3,1,1,12,3,4,2122,1,2,3,3,...,2,1,39,2,1,2,2,2,1,2
4,1,1,12,3,4,2171,1,2,4,3,...,4,2,38,1,2,2,2,1,1,2


In [3]:
df.describe()

,credit.rating,account.balance,credit.duration.months,previous.credit.payment.status,credit.purpose,credit.amount,savings,employment.duration,installment.rate,marital.status,...,residence.duration,current.assets,age,other.credits,apartment.type,bank.credits,occupation,dependents,telephone,foreign.worker
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.00000,1000.000000,1000.000000,...,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.700000,2.183000,20.903000,2.292000,2.965000,3271.24800,1.874000,2.44600,2.973000,2.372000,...,2.845000,2.358000,35.54200,1.814000,1.928000,1.367000,2.904000,1.155000,1.404000,1.037000
std,0.458487,0.835589,12.058814,0.620581,0.971967,2822.75176,1.196476,1.10558,1.118715,1.067125,...,1.103718,1.050209,11.35267,0.389301,0.530186,0.482228,0.653614,0.362086,0.490943,0.188856
min,0.000000,1.000000,4.000000,1.000000,1.000000,250.00000,1.000000,1.00000,1.000000,1.000000,...,1.000000,1.000000,19.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,0.000000,1.000000,12.000000,2.000000,2.000000,1365.50000,1.000000,2.00000,2.000000,1.000000,...,2.000000,1.000000,27.00000,2.000000,2.000000,1.000000,3.000000,1.000000,1.000000,1.000000
50%,1.000000,2.000000,18.000000,2.000000,3.000000,2319.50000,1.000000,2.00000,3.000000,3.000000,...,3.000000,2.000000,33.00000,2.000000,2.000000,1.000000,3.000000,1.000000,1.000000,1.000000
75%,1.000000,3.000000,24.000000,3.000000,4.000000,3972.25000,3.000000,4.00000,4.000000,3.000000,...,4.000000,3.000000,42.00000,2.000000,2.000000,2.000000,3.000000,1.000000,2.000000,1.000000
max,1.000000,3.000000,72.000000,3.000000,4.000000,18424.00000,4.000000,4.00000,4.000000,4.000000,...,4.000000,4.000000,75.00000,2.000000,3.000000,2.000000,4.000000,2.000000,2.000000,2.000000


In [4]:
df.shape

(1000, 21)

In [5]:
array = df.values
array

array([[ 1,  1, 18, ...,  1,  1,  1],
       [ 1,  1,  9, ...,  2,  1,  1],
       [ 1,  2, 12, ...,  1,  1,  1],
       ...,
       [ 0,  3, 21, ...,  1,  2,  1],
       [ 0,  2, 12, ...,  1,  2,  1],
       [ 0,  1, 30, ...,  1,  1,  1]])

In [6]:
x = array[:, 1:]
x

array([[ 1, 18,  3, ...,  1,  1,  1],
       [ 1,  9,  3, ...,  2,  1,  1],
       [ 2, 12,  2, ...,  1,  1,  1],
       ...,
       [ 3, 21,  3, ...,  1,  2,  1],
       [ 2, 12,  2, ...,  1,  2,  1],
       [ 1, 30,  2, ...,  1,  1,  1]])

In [7]:
y = array[:, 0]

In [8]:
test = SelectKBest(score_func=chi2, k=4)
fit = test.fit(x, y)

In [9]:
np.set_printoptions(precision=3)
scores = fit.scores_
print(scores)
features = fit.transform(x)

[3.681e+01 3.210e+02 8.142e+00 3.624e+00 5.826e+04 2.450e+01 6.962e+00
 2.205e+00 3.993e+00 4.357e-05 3.766e-03 9.504e+00 3.018e+01 1.071e+00
 4.782e-02 3.553e-01 1.575e-01 1.031e-03 2.281e-01 2.315e-01]


In [10]:
columns = df.columns
print(features)
print(columns)

[[    1    18  1049    21]
 [    1     9  2799    36]
 [    2    12   841    23]
 ...
 [    3    21 12680    30]
 [    2    12  6468    52]
 [    1    30  6350    31]]
Index(['credit.rating', 'account.balance', 'credit.duration.months',
       'previous.credit.payment.status', 'credit.purpose', 'credit.amount',
       'savings', 'employment.duration', 'installment.rate', 'marital.status',
       'guarantor', 'residence.duration', 'current.assets', 'age',
       'other.credits', 'apartment.type', 'bank.credits', 'occupation',
       'dependents', 'telephone', 'foreign.worker'],
      dtype='object')


In [11]:
data = []
for i in range(len(scores)):
    print('{}: {}'.format(columns[i], scores[i]))

credit.rating: 36.808258621817956
account.balance: 321.0307953062103
credit.duration.months: 8.141693675725104
previous.credit.payment.status: 3.6236248293583877
credit.purpose: 58264.41547539274
credit.amount: 24.503836967017314
savings: 6.9618814001479645
employment.duration: 2.204619351945284
installment.rate: 3.993334939372049
marital.status: 4.3567289678899166e-05
guarantor: 0.0037660055234744255
residence.duration: 9.503534068419562
current.assets: 30.178267964661643
age: 1.071139812043888
other.credits: 0.04781663702825554
apartment.type: 0.3553488696136819
bank.credits: 0.15748393021120405
occupation: 0.0010307153164296021
dependents: 0.22805589472256171
telephone: 0.23148275703724003


In [12]:
# df.drop(columns=['apartment.type', 'telephone', 'dependents',
#                  'bank.credits', 'other.credits', 'guarantor',
#                  'occupation'], inplace=True)
# df.rename(columns={'credit.rating' : 'credit_rating',
# 'account.balance' : 'account_balance',
# 'credit.duration.months' : 'credit_duration_months',
# 'previous.credit.payment.status' : 'previous_credit_payment_status',
# 'credit.purpose' : 'credit_purpose',
# 'credit.amount' : 'credit_amount',
# 'savings' : 'savings',
# 'employment.duration' : 'employment_duration',
# 'installment.rate' : 'installment_rate',
# 'marital.status' : 'marital_status',
# 'residence.duration' : 'residence_duration',
# 'current.assets' : 'current_assets',
# 'age' : 'age',
# 'foreign.worker' : 'foreign_worker'}, inplace=True)
# df.columns

In [13]:
df.drop(columns=['apartment.type', 'telephone', 'dependents',
                 'bank.credits', 'other.credits', 'guarantor',
                 'occupation', 'age'], inplace=True)
df.rename(columns={'credit.rating' : 'credit_rating',
'account.balance' : 'account_balance',
'credit.duration.months' : 'credit_duration_months',
'previous.credit.payment.status' : 'previous_credit_payment_status',
'credit.purpose' : 'credit_purpose',
'credit.amount' : 'credit_amount',
'savings' : 'savings',
'employment.duration' : 'employment_duration',
'installment.rate' : 'installment_rate',
'marital.status' : 'marital_status',
'residence.duration' : 'residence_duration',
'current.assets' : 'current_assets',
'foreign.worker' : 'foreign_worker'}, inplace=True)
df.columns

Index(['credit_rating', 'account_balance', 'credit_duration_months',
       'previous_credit_payment_status', 'credit_purpose', 'credit_amount',
       'savings', 'employment_duration', 'installment_rate', 'marital_status',
       'residence_duration', 'current_assets', 'foreign_worker'],
      dtype='object')

In [14]:
df.head()

,credit_rating,account_balance,credit_duration_months,previous_credit_payment_status,credit_purpose,credit_amount,savings,employment_duration,installment_rate,marital_status,residence_duration,current_assets,foreign_worker
0,1,1,18,3,2,1049,1,1,4,1,4,2,1
1,1,1,9,3,4,2799,1,2,2,3,2,1,1
2,1,2,12,2,4,841,2,3,2,1,4,1,1
3,1,1,12,3,4,2122,1,2,3,3,2,1,2
4,1,1,12,3,4,2171,1,2,4,3,4,2,2


In [15]:
df.describe()

,credit_rating,account_balance,credit_duration_months,previous_credit_payment_status,credit_purpose,credit_amount,savings,employment_duration,installment_rate,marital_status,residence_duration,current_assets,foreign_worker
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.700000,2.183000,20.903000,2.292000,2.965000,3271.24800,1.874000,2.44600,2.973000,2.372000,2.845000,2.358000,1.037000
std,0.458487,0.835589,12.058814,0.620581,0.971967,2822.75176,1.196476,1.10558,1.118715,1.067125,1.103718,1.050209,0.188856
min,0.000000,1.000000,4.000000,1.000000,1.000000,250.00000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,0.000000,1.000000,12.000000,2.000000,2.000000,1365.50000,1.000000,2.00000,2.000000,1.000000,2.000000,1.000000,1.000000
50%,1.000000,2.000000,18.000000,2.000000,3.000000,2319.50000,1.000000,2.00000,3.000000,3.000000,3.000000,2.000000,1.000000
75%,1.000000,3.000000,24.000000,3.000000,4.000000,3972.25000,3.000000,4.00000,4.000000,3.000000,4.000000,3.000000,1.000000
max,1.000000,3.000000,72.000000,3.000000,4.000000,18424.00000,4.000000,4.00000,4.000000,4.000000,4.000000,4.000000,2.000000


In [16]:
df.shape

(1000, 13)

In [17]:
forecasters = df.iloc[:, 1:14].values
forecasters

array([[ 1, 18,  3, ...,  4,  2,  1],
       [ 1,  9,  3, ...,  2,  1,  1],
       [ 2, 12,  2, ...,  4,  1,  1],
       ...,
       [ 3, 21,  3, ...,  4,  4,  1],
       [ 2, 12,  2, ...,  1,  4,  1],
       [ 1, 30,  2, ...,  4,  2,  1]])

In [18]:
classes = df.iloc[:, 0].values

In [19]:
scaler = StandardScaler()
forecasters = scaler.fit_transform(forecasters)
forecasters

/home/martoso/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/martoso/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


array([[-1.416, -0.241,  1.141, ...,  1.047, -0.341, -0.196],
       [-1.416, -0.988,  1.141, ..., -0.766, -1.294, -0.196],
       [-0.219, -0.739, -0.471, ...,  1.047, -1.294, -0.196],
       ...,
       [ 0.978,  0.008,  1.141, ...,  1.047,  1.564, -0.196],
       [-0.219, -0.739, -0.471, ..., -1.672,  1.564, -0.196],
       [-1.416,  0.755, -0.471, ...,  1.047, -0.341, -0.196]])

In [54]:
forecasters_train, forecasters_test, classes_train, classes_test = train_test_split(forecasters
                                                             , classes
                                                             , test_size=0.25
                                                             , random_state=0)

In [55]:
count = collections.Counter(classes_test)
count[0]/(count[0]+count[1])

0.332

In [56]:
estimator_Knn = KNeighborsClassifier(n_neighbors=5, metric='minkowski', p=2)
estimator_Knn.fit(forecasters_train, classes_train)

predictions_Knn = estimator_Knn.predict(forecasters_test)

precision_Knn = accuracy_score(classes_test, predictions_Knn)
print('Precision: {:.2f}%\n'.format(precision_Knn * 100))


matrix = confusion_matrix(classes_test, predictions_Knn)
print('Confusion Matrix:'
  '\n\t0\t1\n0:\t{}\t{}'
  '\n1:\t{}\t{}'.format(matrix[0][0], matrix[0][1], matrix[1][0], matrix[1][1]))

Precision: 76.40%

Confusion Matrix:
	0	1
0:	36	47
1:	12	155


In [57]:
estimator_Random_Forest = RandomForestClassifier(n_estimators=40, criterion="entropy", random_state=0)
estimator_Random_Forest.fit(forecasters_train, classes_train)

predictions_RF = estimator_Random_Forest.predict(forecasters_test)

precision_RF = accuracy_score(classes_test, predictions_RF)
print('Precision: {:.2f}%\n'.format(precision_RF * 100))


matrix = confusion_matrix(classes_test, predictions_RF)
print('Confusion Matrix:'
  '\n\t0\t1\n0:\t{}\t{}'
  '\n1:\t{}\t{}'.format(matrix[0][0], matrix[0][1], matrix[1][0], matrix[1][1]))

Precision: 80.80%

Confusion Matrix:
	0	1
0:	42	41
1:	7	160


In [58]:
estimator_NB = GaussianNB()
estimator_NB.fit(forecasters_train, classes_train)

prediction_NB = estimator_NB.predict(forecasters_test)

precision_NB = accuracy_score(classes_test, prediction_NB)
print('Precision: {:.2f}%\n'.format(precision_NB * 100))


matrix = confusion_matrix(classes_test, prediction_NB)
print('Confusion Matrix:'
  '\n\t0\t1\n0:\t{}\t{}'
  '\n1:\t{}\t{}'.format(matrix[0][0], matrix[0][1], matrix[1][0], matrix[1][1]))

Precision: 76.80%

Confusion Matrix:
	0	1
0:	51	32
1:	26	141
